In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from main import PreProcessing, Statistics
import numpy as np
import json
from scipy.stats import mode, median_abs_deviation, iqr, trim_mean, entropy as ent, skew, kurtosis
from scipy.signal import welch, correlate, stft
from statsmodels.tsa.ar_model import AutoReg
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
from scipy.fft import fft, fftfreq
import entropy as ent

In [2]:
file_path = "C:/Users/antho/Documents/MEMOIRE_M2/audeline_video/Data/A_P_1956-02-21_ON (1).c3d"
# file_path = "C:/Users/antho/Documents/MEMOIRE_M2/CODE_STAGE_M2/DATA_FOG/LE_LIEVRE_Emmanuel_1971_03_19_LEEM1971/2023-05-26/2023-05-26_overlay_detectFOG/Video overlay 15.c3d"
# Définir les informations du patient
patient_id = 1234
date_de_naissance = "1963-06-23"
medicaments = "Traitement X"
# Instancier l'objet DetectFog
detector = PreProcessing(file_path)
detector.creation_json_grace_c3d(patient_id, date_de_naissance, medicaments)
detector.extract_data_interval()
detector.normalize_data() 
data = detector.decoupage_en_fenetres()
detector.label_fenetre()
detector.association_label_fenetre_data()
data = detector.concat_label_fenetre_data()



# On transforme les données en fréquentiel

In [3]:
def transformation_domaine_frequentiel (data, fs = 50):
    # Nombre de points de données par fenêtre
    n = data.shape[1]  # ou 100 si c'est connu

    # Créer un tableau de fréquences
    frequences = fftfreq(n, d=1/fs)
    frequences = frequences[:n//2] # obligé de laisser la data en série, pour générer le graphique des spectres de magnitudes
    frequencies = fftfreq(n, d=1/fs)
    frequencies = frequencies[:n//2]

    # Transposer le tableau de fréquences pour le mettre en colonnes
    frequencies = frequencies.reshape((1, -1))



    # calculer la transformée de Fourier
    fft_result = fft(data, axis = 1)
    fft_magnitudes = np.abs(fft_result)[:,:n//2] # Garder uniquement les valeurs positives, puisque d'après la symétrie de la FFT, les valeurs négatives sont les mêmes que les valeurs positives



    # chemin_fichier_excel_mag = "C:/Users/antho/Documents/MEMOIRE_M2/magnitude_frequence.csv"
    # chemin_fichier_excel_freq = "C:/Users/antho/Documents/MEMOIRE_M2/frequence.csv"

    # # Créer un DataFrame pour stocker les magnitudes des fréquences
    fft_magnitudes = pd.DataFrame(fft_magnitudes)
    frequencies = pd.DataFrame(frequencies)



    # # # Exporter le DataFrame en tant que fichier CSV
    # fft_magnitudes.to_csv(chemin_fichier_excel_mag, index=False)
    # frequencies.to_csv(chemin_fichier_excel_freq, index=False)
    return fft_magnitudes, frequencies


In [4]:
# Parcourir le dictionnaire pour trouver et modifier les DataFrames
def enlever_derniere_ligne_et_colonne_label(data):
    for sensor, sensor_data in data.items():
        if sensor not in ["metadata", "parcours", "FOG"]:
            for side, side_data in sensor_data.items():
                for measure, measure_data in side_data.items():
                    for axis, axis_data in measure_data.items():
                        if isinstance(axis_data, pd.DataFrame):
                            # Afficher le DataFrame avant la modification
                            print(axis_data)
                        
                            # Supprimer la dernière ligne du DataFrame
                            data_moins_derniere_ligne_na = axis_data.drop(axis_data.index[-1])
                            # print(data_moins_derniere_ligne_na)
                            label = data_moins_derniere_ligne_na["label"]
                        
                            # Vérifier si la colonne 'label' existe avant de la supprimer
                            if 'label' in data_moins_derniere_ligne_na.columns:
                                data_moins_colonne_label = data_moins_derniere_ligne_na.drop(columns=["label"])
                                # Mise à jour du DataFrame dans le dictionnaire
                                measure_data[axis] = data_moins_colonne_label
    return data, label

# data, label = enlever_derniere_ligne_et_colonne_label(data)
                            
                            # Afficher le DataFrame après la modification
                        # print(data_moins_colonne_label)


# transofmraiton du tableau clean

In [5]:
# Parcourir le dictionnaire pour trouver et modifier les DataFrames
def enlever_derniere_ligne_et_colonne_label_frequence(data):
    for sensor, sensor_data in data.items():
        if sensor not in ["metadata", "parcours", "FOG"]:
            for side, side_data in sensor_data.items():
                for measure, measure_data in side_data.items():
                    for axis, axis_data in measure_data.items():
                        if isinstance(axis_data, pd.DataFrame):
                        
                            # Supprimer la dernière ligne du DataFrame
                            data_moins_derniere_ligne_na = axis_data.drop(axis_data.index[-1])
                            # print(data_moins_derniere_ligne_na)
                            label = data_moins_derniere_ligne_na["label"]
                        
                            # Vérifier si la colonne 'label' existe avant de la supprimer
                            if 'label' in data_moins_derniere_ligne_na.columns:
                                data_moins_colonne_label = data_moins_derniere_ligne_na.drop(columns=["label"])
                                # Mise à jour du DataFrame dans le dictionnaire
                                measure_data[axis] = data_moins_colonne_label
                                
                                # Calculer la transformée de Fourier
                            fft_magnitudes, frequencies = transformation_domaine_frequentiel(data_moins_colonne_label)
    return data, label,fft_magnitudes, frequencies

data, label, fft_magnitudes, frequencies = enlever_derniere_ligne_et_colonne_label_frequence(data)
                            
                            # Afficher le DataFrame après la modification
                        # print(data_moins_colonne_label)


     label         0         1         2         3         4         5  \
0    noFog  0.005104  0.005385  0.005206  0.005077  0.005123  0.005088   
1    noFog -0.026491 -0.025612 -0.008221  0.008545  0.017730  0.026927   
2    noFog  0.006660  0.008086  0.008420  0.006764  0.006344  0.006144   
3    noFog  0.006554  0.006837  0.005879  0.005389  0.006014  0.005986   
4    noFog  0.006355  0.005917  0.005917  0.006527  0.006293  0.005757   
..     ...       ...       ...       ...       ...       ...       ...   
153  noFog  0.005659  0.005014  0.004286  0.006275  0.005482  0.005085   
154  noFog  0.004120  0.003730  0.003939  0.004320  0.004915  0.005427   
155  noFog  0.005782  0.005545  0.005608  0.005860  0.005844  0.005743   
156  noFog  0.017938  0.046274  0.089144  0.099361  0.095361  0.060437   
157  noFog  0.005872  0.006387  0.006124  0.006305  0.007063  0.006500   

            6         7         8  ...        90        91        92  \
0    0.005219  0.005450  0.005318  ... 

In [11]:
a = data["Foot"]["Left"]["ACC"]["Y"]

In [12]:
fft_magnitudes, frequencies = transformation_domaine_frequentiel(a, 50)

# J'ai un tableau tout propre

In [13]:
def calcul_entropie_spectrale (fft_magnitudes):
    # Calculer l'entropie spectrale de puissance pour chaque fenêtre
    entropie_spectrale = []

    for index, row in fft_magnitudes.iterrows():
        # Calculer les proportions pi de la puissance spectrale
        puissance_totale = np.sum(row**2)
        p_i = (row**2) / puissance_totale
    
        # Filtrer les valeurs de p_i égales à 0 pour éviter les erreurs de log(0)
        p_i = p_i[p_i > 0]

        # Calculer l'entropie spectrale pour la fenêtre actuelle
        H = -np.sum(p_i * np.log(p_i))
        entropie_spectrale.append(H)

    # Convertir la liste d'entropie en un tableau numpy pour une manipulation facile
    df_entropie_spectrale = pd.DataFrame({'Entropie_Spectrale': entropie_spectrale})
    return df_entropie_spectrale

entropie_spectrale = calcul_entropie_spectrale(fft_magnitudes)

In [22]:
def calcul_details_harmoniques (fft_magnitudes, frequencies):

    # Initialiser les listes pour stocker les résultats
    premiere_harmonique_mag = []
    deuxieme_harmonique_mag = []
    premiere_harmonique_freq = []
    deuxieme_harmonique_freq = []
    distance_harmonique_frequence = []  # Liste pour stocker la distance entre les harmoniques
    distance_harmonique_magnitude = []
    centre_densite_spectrale = []
    centre_densite_spectrale_puissance = []
    rapport_harmonique_frequence = []
    rapport_harmonique_magnitude = []
    crete_spectrale_puissance_ponderee_gpt = []
    crete_spectrale_puissance_ponderee_borzi = []
    largeurs_harmoniques = []


    # Itérer sur chaque fenêtre
    for index, row in fft_magnitudes.iterrows():
        magnitudes = row.values
        frequences = frequencies.values.flatten() # Assumer que les fréquences sont constantes et identiques pour toutes les fenêtres
    
        # Trouver les indices des deux plus grandes magnitudes
        indices_harmoniques = np.argsort(magnitudes)[-2:]  # Cela nous donne les indices du second puis du premier
    
        # Assurer que l'indice de la première harmonique est celui de la plus grande magnitude
        if magnitudes[indices_harmoniques[0]] > magnitudes[indices_harmoniques[1]]:
            premiere_harmonique, deuxieme_harmonique = indices_harmoniques[0], indices_harmoniques[1]
        else:
            premiere_harmonique, deuxieme_harmonique = indices_harmoniques[1], indices_harmoniques[0]
    
        # Calculer le centre de densité spectrale (CDS)
        cds = np.sum(frequences * magnitudes) / np.sum(magnitudes)
    
        # Calculer le centre de densité spectrale de puissance
        cds_puissance = np.sum(frequences * magnitudes**2) / np.sum(magnitudes**2)
    
        # Calcul de la crête spectrale de puissance pondérée selon GPT
        cs_puissance_ponderee_gpt = np.max(magnitudes**2) / np.sum(magnitudes**2)
    
        # Calcul de la crête spectrale de puissance pondérée selon Borzi
        cs_puissance_ponderee_borzi = ((magnitudes[premiere_harmonique]**2) * frequences[premiere_harmonique])
    
        # Stocker les résultats
        premiere_harmonique_mag.append(magnitudes[premiere_harmonique])
        deuxieme_harmonique_mag.append(magnitudes[deuxieme_harmonique])
        premiere_harmonique_freq.append(frequences[premiere_harmonique])
        deuxieme_harmonique_freq.append(frequences[deuxieme_harmonique])
        centre_densite_spectrale.append(cds)
        centre_densite_spectrale_puissance.append(cds_puissance)
        crete_spectrale_puissance_ponderee_gpt.append(cs_puissance_ponderee_gpt)
        crete_spectrale_puissance_ponderee_borzi.append(cs_puissance_ponderee_borzi)
    
        # Calculer et stocker la distance de fréquence entre les harmoniques
        distance_harmonique_frequence.append(abs(frequences[premiere_harmonique] - frequences[deuxieme_harmonique]))
    
    
        # Pour éviter Inf, vérifier si le dénominateur est zéro
        if frequences[deuxieme_harmonique] == 0:
            rapport_harmonique_frequence.append(0)
        else:
            rapport_harmonique_frequence.append(frequences[premiere_harmonique] / frequences[deuxieme_harmonique])
    
    
        # Calculer et stocker la distance de magnitude entre les harmoniques
        distance_harmonique_magnitude.append(abs(magnitudes[premiere_harmonique] - magnitudes[deuxieme_harmonique]))
    
        # De même, éviter Inf pour les magnitudes
        if magnitudes[deuxieme_harmonique] == 0:
            rapport_harmonique_magnitude.append(0)
        else:
            rapport_harmonique_magnitude.append(magnitudes[premiere_harmonique] / magnitudes[deuxieme_harmonique])
        
        
           
        # Calculer la largeur des harmoniques
            # Calculer la magnitude de la première harmonique
        premiere_harmonique_magnitude = magnitudes[premiere_harmonique]
    
        # Utiliser la bonne méthode pour trouver les indices gauche et droite
        # Trouver l'indice de gauche
        gauche = np.where(magnitudes[:premiere_harmonique] < premiere_harmonique_magnitude * 0.5)[0]
        if len(gauche) > 0:
            indice_gauche = gauche[-1] + 1  # Prendre le dernier indice sous le seuil et ajouter 1
        else:
            indice_gauche = 0  # S'il n'y a pas de valeur sous le seuil, prendre le début du signal
    
        # Trouver l'indice de droite
        droite = np.where(magnitudes[premiere_harmonique+1:] < premiere_harmonique_magnitude * 0.5)[0]
        if len(droite) > 0:
            indice_droite = droite[0] + premiere_harmonique + 1  # Prendre le premier indice sous le seuil après le pic
        else:
            indice_droite = len(magnitudes) - 1  # S'il n'y a pas de valeur sous le seuil, prendre la fin du signal
    
        # Calculer la largeur en Hz
        largeur_hz = frequences[indice_droite] - frequences[indice_gauche]
        largeurs_harmoniques.append(largeur_hz)

    # Créer un DataFrame pour les résultats
    df_resultats = pd.DataFrame({
        'Premiere_Harmonique_Magnitude': premiere_harmonique_mag,
        'Deuxieme_Harmonique_Magnitude': deuxieme_harmonique_mag,
        'Premiere_Harmonique_Frequence': premiere_harmonique_freq,
        'Deuxieme_Harmonique_Frequence': deuxieme_harmonique_freq,
        'Distance_Harmonique_Frequence': distance_harmonique_frequence,
        'Distance_Harmonique_Amplitude':  distance_harmonique_magnitude,
        'Rapport_Harmonique_Frequence': rapport_harmonique_frequence,
        'Rapport_Harmonique_Amplitude':  rapport_harmonique_magnitude,
        'Centre_Densite_Spectrale': centre_densite_spectrale,
        'Centre_Densite_Spectrale_Puissance': centre_densite_spectrale_puissance,
        'Crete_Spectrale_Puissance_Ponderee_GPT': crete_spectrale_puissance_ponderee_gpt,
        'Crete_Spectrale_Puissance_Ponderee_Borzi': crete_spectrale_puissance_ponderee_borzi,
        'Largeur_Harmonique': largeurs_harmoniques
        })
    
    return df_resultats
    # return premiere_harmonique_mag,deuxieme_harmonique_mag,premiere_harmonique_freq,deuxieme_harmonique_freq, distance_harmonique_frequence,distance_harmonique_magnitude,rapport_harmonique_frequence,rapport_harmonique_magnitude, centre_densite_spectrale,centre_densite_spectrale_puissance, crete_spectrale_puissance_ponderee_gpt, crete_spectrale_puissance_ponderee_borzi, largeurs_harmoniques

# # Afficher les premières lignes du DataFrame résultant pour vérification
# print(df_resultats.head())
# df_resultats = calcul_details_harmoniques(fft_magnitudes, frequencies)


In [23]:
def ecart_type_borne (fft_magnitudes, frequences):
    # Définissons les bandes de fréquences spécifiées
    bandes_frequence = {
        'ecart_type': (0, 50),
        'ecart_type_0.04_0.68_Hz': (0.04, 0.68),
        'ecart_type_0.68_3_Hz': (0.68, 3),
        'ecart_type_3_8_Hz': (3, 8),
        'ecart_type_8_20_Hz': (8, 20),
        'ecart_type_0.1_8_Hz': (0.1, 8)
    }

    # Extrayons les fréquences depuis le fichier frequence.csv pour l'associer à chaque colonne de magnitude_frequence.csv
    frequences = frequencies.values.flatten()

    # Créons un DataFrame pour stocker les écarts types calculés pour chaque bande de fréquence et pour chaque ligne (fenêtre)
    ecarts_types = pd.DataFrame()

    # Pour chaque bande de fréquence, filtrons les données et calculons l'écart type
    for nom_bande, (freq_min, freq_max) in bandes_frequence.items():
    
        # Identifions les colonnes correspondant à la bande de fréquence
        colonnes_bande = (frequences >= freq_min) & (frequences <= freq_max)
    
        # Filtrons les magnitudes pour cette bande de fréquence
        magnitudes_bande = fft_magnitudes.loc[:, colonnes_bande]
    
        # Calculons l'écart type pour cette bande de fréquence et ajoutons les résultats au DataFrame
        ecarts_types[nom_bande] = magnitudes_bande.std(axis=1)
        
    return ecarts_types

# # # Affichons les premières lignes des résultats pour vérifier
# ecarts_type = ecart_type_borne(fft_magnitudes, frequencies)

In [24]:
def calculer_freeze_index(magnitudes, frequences):
    """
    Calcule le Freeze Index pour chaque fenêtre de données.
    
    :param magnitudes: Un DataFrame ou un numpy array des magnitudes du spectre de puissance pour chaque fenêtre.
    :param frequences: Un numpy array des fréquences correspondant aux colonnes de magnitudes.
    :return: Un numpy array contenant le Freeze Index pour chaque fenêtre.
    """
    
    # Définissons une fonction interne pour calculer l'aire sous le spectre de puissance
    def calculer_aire_sous_spectre(frequences, magnitudes, freq_min, freq_max):
        indices_bande = (frequences >= freq_min) & (frequences <= freq_max)
        magnitudes_bande = magnitudes[:, indices_bande]
        aire_sous_spectre = np.trapz(magnitudes_bande, x=frequences[indices_bande], axis=1)
        return aire_sous_spectre

    # Bandes de fréquences pour le Freeze Index
    bande_freeze = (3, 8)  # Bande "freeze"
    bande_locomotrice = (0.5, 3)  # Bande "locomotrice"

    # Calcul de l'aire sous le spectre pour chaque bande
    aire_freeze = calculer_aire_sous_spectre(frequences, magnitudes, *bande_freeze)
    aire_locomotrice = calculer_aire_sous_spectre(frequences, magnitudes, *bande_locomotrice)

    # Calcul du Freeze Index
    freeze_index = (aire_freeze ** 2) / (aire_locomotrice ** 2)

    return freeze_index

# Exemple d'utilisation de la fonction `calculer_freeze_index`
# Assurez-vous que `magnitude_frequence_df` et `frequences` sont définis et chargés correctement
# magnitudes = fft_magnitudes.values  # Convertissons le DataFrame en numpy array si ce n'est pas déjà le cas
# frequences = frequencies.values.flatten()

# Calculons le Freeze Index
# freeze_index_resultats = calculer_freeze_index(magnitudes, frequences)

# # Créons un DataFrame pour afficher les Freeze Index calculés
# freeze_index_df = pd.DataFrame({'Freeze Index': freeze_index_resultats})

# # Affichage des premiers résultats
# print(freeze_index_df.head())

In [9]:
## Fréquence de faible Puissance pour une bande fréquence entre 0 et 2 Hz
def ratio_faible_puissance_entre_0_2Hz (fft_magnitudes,frequencies):
    # Assurez-vous que `magnitude_frequence_df` et `frequences` sont définis et chargés correctement
    magnitudes = fft_magnitudes.values  # Convertissons le DataFrame en numpy array si ce n'est pas déjà le cas
    frequences = frequencies.values.flatten()

    ratios = []  # Pour stocker le ratio de chaque fenêtre
    psd = np.abs(magnitudes)**2  # Calcul de la densité spectrale de puissance
    puissance_totale = np.sum(psd, axis=1)  # Calcul de la puissance totale du signal pour chaque fenêtre
    
    # Filtrer pour obtenir la puissance dans la bande 0-2 Hz
    bande_indices = (frequences >= 0) & (frequences <= 2)
    psd_band= psd[:, bande_indices]
    puissance_bande = np.sum(psd_band, axis = 1)
        
    # Calculer le ratio pour la fenêtre
    for fenetre in magnitudes:
        ratio_puissance_bande = puissance_bande / puissance_totale
        ratios.append(ratio_puissance_bande)
        
    return ratios

# Exemple d'utilisation de la fonction `ratio_faible_puissance_entre_0_2Hz`
# ratios = ratio_faible_puissance_entre_0_2Hz(fft_magnitudes, frequencies)


In [10]:
def skewness_band_freq (fft_magnitudes,frequencies):
    magnitudes = fft_magnitudes.values  # Convertissons le DataFrame en numpy array si ce n'est pas déjà le cas
    frequences = frequencies.values.flatten()


    # Définissons les bandes de fréquences spécifiées
    bandes_frequence = {
        'skewness': (0, 50),
        'skewness_0.04_0.68_Hz': (0.04, 0.68),
        'skewness_0.68_3_Hz': (0.68, 3),
        'skewness_3_8_Hz': (3, 8),
        'skewness_8_20_Hz': (8, 20),
        'skewness_0.1_8_Hz': (0.1, 8)
    }

    # Créons un DataFrame pour stocker les écarts types calculés pour chaque bande de fréquence et pour chaque ligne (fenêtre)
    skwenesss = pd.DataFrame()

    # Pour chaque bande de fréquence, filtrons les données et calculons l'écart type
    for nom_bande, (freq_min, freq_max) in bandes_frequence.items():
    
        # Identifions les colonnes correspondant à la bande de fréquence
        colonnes_bande = (frequences >= freq_min) & (frequences <= freq_max)
    
        # Filtrons les magnitudes pour cette bande de fréquence
        magnitudes_bande = fft_magnitudes.loc[:, colonnes_bande]
    
        # Calculons l'écart type pour cette bande de fréquence et ajoutons les résultats au DataFrame
        skwenesss[nom_bande] = magnitudes_bande.skew(axis=1)

    # Affichons les premières lignes des résultats pour vérifier
    # skwenesss.head()
    return skwenesss

# skewnesss = skewness_band_freq(fft_magnitudes, frequencies)
# print(skewnesss)


In [11]:
def kurtosis_band_freq (fft_magnitudes, frequencies) :
    magnitudes = fft_magnitudes.values  # Convertissons le DataFrame en numpy array si ce n'est pas déjà le cas
    frequences = frequencies.values.flatten()
    # Définissons les bandes de fréquences spécifiées
    bandes_frequence = {
        'kurtosis': (0, 50),
        'kurtosis_0.04_0.68_Hz': (0.04, 0.68),
        'kurtosis_0.68_3_Hz': (0.68, 3),
        'kurtosis_3_8_Hz': (3, 8),
        'kurtosis_8_20_Hz': (8, 20),
        'kurtosis_0.1_8_Hz': (0.1, 8)
    }

    # Créons un DataFrame pour stocker les écarts types calculés pour chaque bande de fréquence et pour chaque ligne (fenêtre)
    kurtosiss = pd.DataFrame()

    # Pour chaque bande de fréquence, filtrons les données et calculons l'écart type
    for nom_bande, (freq_min, freq_max) in bandes_frequence.items():
    
        # Identifions les colonnes correspondant à la bande de fréquence
        colonnes_bande = (frequences >= freq_min) & (frequences <= freq_max)
    
        # Filtrons les magnitudes pour cette bande de fréquence
        magnitudes_bande = fft_magnitudes.loc[:, colonnes_bande]
    
        # Calculons l'écart type pour cette bande de fréquence et ajoutons les résultats au DataFrame
        kurtosiss[nom_bande] = magnitudes_bande.kurtosis(axis=1)

    # Affichons les premières lignes des résultats pour vérifier
    kurtosiss.head()
    
    return kurtosiss

# kurtosisss = kurtosis_band_freq(fft_magnitudes,frequencies)



In [12]:
def calcul_locomotion_band_power (fft_magnitudes,frequencies):
# Locomotion band power
    magnitudes = fft_magnitudes.values  # Convertissons le DataFrame en numpy array si ce n'est pas déjà le cas
    frequences = frequencies.values.flatten()
    # Filtrer pour obtenir la puissance dans la bande de locomotion (0.5-3 Hz)
    bande_locomotion_power_list = []
    psd = np.abs(magnitudes)**2 
    bande_locomotion = (frequences >= 0.5) & (frequences <= 3)
    psd_bande_locomotion = psd[:, bande_locomotion]
    puissance_bande_locomotion = np.sum(psd_bande_locomotion, axis=1)

    for window in puissance_bande_locomotion:
        bande_locomotion_power = window / 50
        bande_locomotion_power_list.append(bande_locomotion_power)
        
    return bande_locomotion_power

# bande_locomotion_power = calcul_locomotion_band_power (fft_magnitudes,frequencies)

In [12]:
def calcul_freeze_band_power (fft_magnitudes,frequencies):
    magnitudes = fft_magnitudes.values  # Convertissons le DataFrame en numpy array si ce n'est pas déjà le cas
    frequences = frequencies.values.flatten()
    # Filtrer pour obtenir la puissance dans la bande de locomotion (3-8 Hz)
    bande_freeze_power_list = []
    psd = np.abs(magnitudes)**2 


    bande_freeze = (frequences >= 3) & (frequences <= 8)
    psd_bande_freeze = psd[:, bande_freeze]
    puissance_bande_freeze = np.sum(psd_bande_freeze, axis=1)

    for window in puissance_bande_freeze:
        bande_freeze_power = window / 50
        bande_freeze_power_list.append(bande_freeze_power)
        
    return bande_freeze_power

In [13]:
def calcul_band_power(fft_magnitudes,frequencies):
# Locomotion band power
    magnitudes = fft_magnitudes.values  # Convertissons le DataFrame en numpy array si ce n'est pas déjà le cas
    frequences = frequencies.values.flatten()
    bande_power_list = []
    psd = np.abs(magnitudes)**2 

    # Filtrer pour obtenir la puissance dans la bande de locomotion et de freezing (0.5-8 Hz)
    bande_power = (frequences >= 0.5) & (frequences <= 8)
    psd_bande_power = psd[:, bande_power]
    puissance_bande_power= np.sum(psd_bande_power, axis=1)

    for window in puissance_bande_power:
        bande_power = window / 50
        bande_power_list.append(bande_power)
    return bande_power



In [14]:
def calcul_energie (fft_magnitudes, frequencies):
    magnitudes = fft_magnitudes.values  # Convertissons le DataFrame en numpy array si ce n'est pas déjà le cas
    frequences = frequencies.values.flatten()
    energie_liste = []
    
    for window in magnitudes:
        # Calculer l'énergie de la fenêtre comme la somme des carrés des valeurs du signal divisée par la longueur de la fenêtre
        energie = np.sum(np.square(magnitudes)) / len(window)
        energie_liste.append(energie)
        
    return energie

# Extraction finale de toutes les caractéristiques

In [42]:
# Parcourir le dictionnaire pour trouver et modifier les DataFrames
def dataframe_caracteristiques_final(data):
    df_final = pd.DataFrame()

    for sensor, sensor_data in data.items():
        if sensor not in ["metadata", "parcours", "FOG"]:
            for side, side_data in sensor_data.items():
                for measure, measure_data in side_data.items():
                    for axis, axis_data in measure_data.items():
                        if isinstance(axis_data, pd.DataFrame):
                            # Application des fonctions pour calculer les caractéristiques
                            fft_magnitude, frequencies = transformation_domaine_frequentiel(axis_data, fs=50)
                            entropie_spectrale = calcul_entropie_spectrale(fft_magnitude)
                            details_harmoniques = calcul_details_harmoniques(fft_magnitude, frequencies)
                            ecart_types = ecart_type_borne(fft_magnitude, frequencies)
                            
                            
                            
                            
                            
                            # Fusionner toutes les caractéristiques dans un seul DataFrame pour simplification
                            caract_features = pd.concat([entropie_spectrale, details_harmoniques, ecart_types], axis=1)

                            # Création dynamique des noms de colonnes pour chaque caractéristique
                            for feature_name in caract_features.columns:
                                column_name = f"{sensor}_{side}_{measure}_{axis}_{feature_name}"
                                # Ajout de la caractéristique avec le nouveau nom de colonne dans le DataFrame final
                                if column_name not in df_final:
                                    df_final[column_name] = caract_features[feature_name]
                                else:
                                    # Si la colonne existe déjà, on ajoute les données en ligne
                                    df_final[column_name] = pd.concat([df_final[column_name], caract_features[feature_name]], ignore_index=True)
                                
                                
                                
                                
                                
                                
    return df_final

df_final = dataframe_caracteristiques_final(data)





C:\Users\antho\AppData\Local\Temp\ipykernel_18208\2795698186.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[column_name] = caract_features[feature_name]
C:\Users\antho\AppData\Local\Temp\ipykernel_18208\2795698186.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[column_name] = caract_features[feature_name]
C:\Users\antho\AppData\Local\Temp\ipykernel_18208\2795698186.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has po

In [30]:
chemin_fichier_csv_data_final = "C:/Users/antho/Documents/MEMOIRE_M2/data_final.csv"




    # # # Exporter le DataFrame en tant que fichier CSV
df_final.to_csv(chemin_fichier_csv_data_final, index=False)

In [43]:
# def dataframe_caracteristiques_final(data):
#     data_collect = []

#     for sensor, sensor_data in data.items():
#         if sensor not in ["metadata", "parcours", "FOG"]:
#             for side, side_data in sensor_data.items():
#                 for measure, measure_data in side_data.items():
#                     for axis, axis_data in measure_data.items():
#                         if isinstance(axis_data, pd.DataFrame):
#                             fft_magnitude, frequencies = transformation_domaine_frequentiel(axis_data, fs=50)
#                             entropie_spectrale = calcul_entropie_spectrale(fft_magnitude)
#                             details_harmoniques = calcul_details_harmoniques(fft_magnitude, frequencies)
#                             ecart_types = ecart_type_borne(fft_magnitude, frequencies)

#                             caract_features = pd.concat([entropie_spectrale, details_harmoniques, ecart_types], axis=1)
#                             # print(caract_features)
#                             # caract_features.reset_index(drop=True, inplace=True)
#                             for feature_name in caract_features.columns:
#                                 caract_features.rename(columns={feature_name: f"{sensor}_{side}_{measure}_{axis}_{feature_name}"})
#                             data_collect.append(caract_features)

#     df_final = pd.concat(data_collect, axis=1, ignore_index=True)
#     return df_final

# df_final = dataframe_caracteristiques_final(data)

# CODE FINAL

In [44]:
def dataframe_caracteristiques_final(data):
    data_collect = []

    for sensor, sensor_data in data.items():
        if sensor not in ["metadata", "parcours", "FOG"]:
            for side, side_data in sensor_data.items():
                for measure, measure_data in side_data.items():
                    for axis, axis_data in measure_data.items():
                        if isinstance(axis_data, pd.DataFrame):
                            fft_magnitude, frequencies = transformation_domaine_frequentiel(axis_data, fs=50)
                            entropie_spectrale = calcul_entropie_spectrale(fft_magnitude)
                            details_harmoniques = calcul_details_harmoniques(fft_magnitude, frequencies)
                            ecart_types = ecart_type_borne(fft_magnitude, frequencies)

                            caract_features = pd.concat([entropie_spectrale, details_harmoniques, ecart_types], axis=1)

                            # Renommer les colonnes ici
                            caract_features.rename(columns={feature_name: f"{sensor}_{side}_{measure}_{axis}_{feature_name}" for feature_name in caract_features.columns}, inplace=True)
                            
                            data_collect.append(caract_features)

    # Concaténer toutes les données collectées en alignant les colonnes
    df_final = pd.concat(data_collect, axis=1)
    return df_final

# Supposons que `data` est déjà défini et structuré correctement
df_final = dataframe_caracteristiques_final(data)
